*想要将深度学习用于小型图像数据集，需使用已在大型数据集训练好保存好的预训练网络。深度学习从数据中进行学习有两个基本特征：第一，通过渐进的、逐层的方式形成越来越复杂的表示；第二，对中间这些渐进的表示共同进行学习，每层变化都需要同时考虑上下两层的需要。*

使用预训练的CNN即在大型数据集上训练好的应用于小型图像数据集，，本例使用的是VGG16（ImageNet数据集上训练好的网络）
### 使用预训练网络：特征提取，微调模型
特征提取：去除之前训练好的网络的卷积基，在上面运行新数据，然后再输出上面训练一个新的分类器。（用于分类图像的CNN包括两部分：首先一系列卷积层和池化层，其次密集连接分类器）



分类器学到的表示必然是针对模型训练的类别（某个类别出现在整张图像的概率信息。
某个卷积层提取的表示的通用性取决于该层在模型中的深度。即模型中靠近底部的层（定义模型时先添加到模型中的层）提取的是局部的、高度通用的特征图（如颜色，纹理视觉边缘等），靠近顶部的层（后添加到模型中的层）提取的是更抽象的概念（如猫耳朵），所以，若新数据与原始模型的数据集差异较大，最好选取模型的前几层来做特征提取，而非使用整个卷积基。





In [0]:
#将VGG16卷积基实例化
import keras
from keras.applications import VGG16     #该模型内置于keras中

conv_base = VGG16(weights='imagenet',        #模型初始化的权重检查点
          include_top=False,          #指定模型最后是否包含密集连接分类器
          input_shape=(150,150,3))    #输入到网络中的图像张量的形状
conv_base.summary()

Using TensorFlow backend.






58892288/58889256 [==============================] - 2s 0us/step






Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
____________________________________________________

最后的特征图形状为（4，4，512），在此特征上添加一个密集连接分类器。